In [29]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\trinh\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy.linalg as LA

C:\Users\trinh\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [3]:
box = (120, 10, 400, 470)

In [4]:
trigger = {
    "dung" : False,
    "ngoi" : False,
    "nghieng phai" : False,
    "nghieng trai" : False,
    "nhay" : False,
    "nhay thap" : False
}

In [21]:
def check_all_in_box(frame, keypoints, box, update_start):
        y, x, c = frame.shape
        shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

        body_point = [shaped[5], shaped[6], shaped[11], shaped[12]]

        th = 0.3
        if update_start is True:
            th = 0.4

        for kp in body_point:
            y, x, conf = kp
            if conf < th:
                return False

        if shaped[6][1] < box[0]:
            return False
        if shaped[5][1] > box[0] + box[2]:
            return False
        if shaped[0][0] > box[1] + box[3] * 2/3:
            return False

        return True

In [22]:
def show_pose(pose):
    cv2.putText(frame, pose, (190, 30), 3, 1, (0, 0, 255), 2, cv2.LINE_AA)  

In [23]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [24]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [25]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [26]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0)
    draw_keypoints(frame, keypoints_with_scores, 0)
    
    # Check Inside Box
    color = ()
    isInside = False
    
    if check_all_in_box(frame, keypoints_with_scores, box, False):
        isInside = True
        color = (0, 255, 0)
    else:
        isInside = False
        color = (0, 0, 255)
    
    # Draw Box
    cv2.line(frame, (box[0], box[1]), (box[0], box[1] + box[3]), color, 2)
    cv2.line(frame, (box[0], box[1]), (box[0] + box[2], box[1]), color, 2)
    cv2.line(frame, (box[0] + box[2], box[1] + box[3]), (box[0] + box[2], box[1]), color, 2)
    cv2.line(frame, (box[0] + box[2], box[1] + box[3]), (box[0], box[1]+box[3]), color, 2)
    
    # Crop and Show
#     frame_resize = frame[0:480, 160:480]
    cv2.imshow('MoveNet Lightning', frame)
    
    # Close
    while cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
    if cv2.getWindowProperty('MoveNet Lightning', 4) < 1:
        break
                                                                                                 
cap.release()
cv2.destroyAllWindows()
